# Анализ A/B-эксперимента рекомендательной системы

## Цель исследования

Оценить, улучшает ли тестовая модель рекомендаций пользовательскую вовлечённость по сравнению с контрольной моделью.

# 1. Импорт библиотек

In [1]:
import numpy as np
import pandas as pd
import hashlib

from scipy.stats import binomtest, mannwhitneyu, ttest_ind
from statsmodels.stats.proportion import proportions_ztest

# 2. Загрузка данных

In [2]:
%%time

views = pd.read_csv('views.csv')
likes = pd.read_csv('likes.csv')


display(views.info(),
        likes.info())

display(views,
        likes)

<class 'pandas.DataFrame'>
RangeIndex: 193295 entries, 0 to 193294
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   user_id          193295 non-null  int64
 1   exp_group        193295 non-null  str  
 2   recommendations  193295 non-null  str  
 3   timestamp        193295 non-null  int64
dtypes: int64(2), str(2)
memory usage: 5.9 MB
<class 'pandas.DataFrame'>
RangeIndex: 230176 entries, 0 to 230175
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    230176 non-null  int64
 1   post_id    230176 non-null  int64
 2   timestamp  230176 non-null  int64
dtypes: int64(3)
memory usage: 5.3 MB


None

None

,user_id,exp_group,recommendations,timestamp
0,128381,control,[3644 4529 4704 5294 4808],1654030803
1,146885,test,[1399 1076 797 7015 5942],1654030811
2,50948,test,[2315 3037 1861 6567 4093],1654030825
3,37703,test,[2842 1949 162 1588 6794],1654030826
4,14661,test,[2395 5881 5648 3417 673],1654030829
...,...,...,...,...
193290,158267,test,[1733 6834 4380 1915 1627],1655240340
193291,63527,control,[2454 191 3873 6404 1588],1655240347
193292,52169,test,[1368 1709 1616 798 5305],1655240354
193293,142402,test,[5895 6984 1978 6548 6106],1655240373


,user_id,post_id,timestamp
0,128381,4704,1654030804
1,146885,1399,1654030816
2,50948,2315,1654030828
3,14661,673,1654030831
4,37703,1588,1654030833
...,...,...,...
230171,31851,5964,1655243535
230172,51512,1498,1655243537
230173,34017,5009,1655243573
230174,13267,1787,1655243692


CPU times: total: 453 ms
Wall time: 512 ms


# 3. Проверка консистентности групп

Нужно:

1. Проверить, нет ли пользователей, которые попали и в `control`, и в `test`.
2. Если такие есть — удалить их полностью.
3. Построить таблицу `user_id → exp_group`.
4. Проверить равномерность 50/50.
5. Применить биномиальный тест.

In [3]:
# Для каждого пользователя считаем количество уникальных групп
user_group_counts = (
    views
    .groupby("user_id")["exp_group"]
    .nunique()
    .reset_index(name="n_groups")
)

# Пользователи, попавшие более чем в одну группу
conflicted_users = user_group_counts[user_group_counts["n_groups"] > 1]

print(f"Количество пользователей, попавших в обе группы: {len(conflicted_users)}")
display(conflicted_users)

Количество пользователей, попавших в обе группы: 4


,user_id,n_groups
10071,25623,2
20633,55788,2
54475,142283,2
57065,148670,2


# 4. Удаляем конфликтных пользователей

In [4]:
# Удаляем конфликтных пользователей из выборки
clean_views = views[~views["user_id"].isin(conflicted_users["user_id"])].copy()
print(f"Размер данных после очистки: {clean_views.shape}")

Размер данных после очистки: (193268, 4)


# 5. Строим таблицу соответствия user → группа

Теперь гарантированно у пользователя только одна группа.

In [5]:
# Строим таблицу user_id → exp_group
user_groups = (
    clean_views
    .groupby("user_id")["exp_group"]
    .first()
    .reset_index()
)

display(user_groups)

,user_id,exp_group
0,200,test
1,201,test
2,202,test
3,212,test
4,213,test
...,...,...
65004,168538,test
65005,168541,control
65006,168544,test
65007,168545,test


# 6. Проверка равномерности распределения

Посмотрим пропорции:

In [6]:
group_distribution = (
    user_groups["exp_group"]
    .value_counts(normalize=True)
)

group_counts = user_groups["exp_group"].value_counts()

print("Количество пользователей в группах:")
display(group_counts)

print("\nДоли групп:")
display(group_distribution)

Количество пользователей в группах:


exp_group
test       32659
control    32350
Name: count, dtype: int64


Доли групп:


exp_group
test       0.502377
control    0.497623
Name: proportion, dtype: float64

# 7. Биномиальный тест (проверка 50/50)

Теперь проверим статистически: соответствует ли доля контрольной группы ожидаемым 0.5.

In [7]:
n_total = len(user_groups)
n_control = group_counts.get("control", 0)

# Биномиальный тест: H0 — доля control = 0.5
binom_result = binomtest(
    k=n_control,
    n=n_total,
    p=0.5,
    alternative="two-sided"
)

print("Биномиальный тест на соответствие 50/50:")
print(f"p-value = {binom_result.pvalue:.6f}")

Биномиальный тест на соответствие 50/50:
p-value = 0.227050


# 📌 Как интерпретировать

* **p-value > 0.05** → разбиение соответствует 50/50
* **p-value < 0.05** → статистически значимое отклонение

# 8. Подготовка таблицы пользователей эксперимента

In [8]:
# Пользователи, участвовавшие в эксперименте
experiment_users = user_groups[["user_id"]].copy()

print(f"Количество пользователей в эксперименте: {len(experiment_users)}")
display(experiment_users)

Количество пользователей в эксперименте: 65009


,user_id
0,200
1,201
2,202
3,212
4,213
...,...
65004,168538
65005,168541
65006,168544
65007,168545


# 9. Считаем количество лайков на пользователя

In [9]:
user_likes = (
    likes
    .groupby("user_id")
    .size()
    .reset_index(name="n_likes")
)

display(user_likes)

,user_id,n_likes
0,200,1
1,201,3
2,202,2
3,212,4
4,213,7
...,...,...
58166,168525,2
58167,168532,4
58168,168538,5
58169,168541,2


# 10. Объединяем пользователей и лайки

Важно: используем `left join`, чтобы сохранить пользователей без лайков.

In [10]:
# Объединяем данные
user_activity = (
    experiment_users
    .merge(user_likes, on="user_id", how="left")
)

# Заполняем пропуски нулями (если лайков не было)
user_activity["n_likes"] = user_activity["n_likes"].fillna(0).astype(int)

display(user_activity)

,user_id,n_likes
0,200,1
1,201,3
2,202,2
3,212,4
4,213,7
...,...,...
65004,168538,5
65005,168541,2
65006,168544,0
65007,168545,5


# 11. Доля пользователей с ≥1 лайком

In [11]:
# Доля пользователей, сделавших хотя бы один лайк
share_with_like = (user_activity["n_likes"] > 0).mean()

print(f"Доля пользователей с ≥1 лайком: {share_with_like:.4f}")

Доля пользователей с ≥1 лайком: 0.8948


# Размышления

## Метрика 1:

### Доля пользователей с ≥1 лайком

* это бинарная переменная (0/1)  
* сравниваем **две доли**  
* подходит **z-test для пропорций**  

## Метрика 2:

### Число лайков на пользователя

* это дискретная счётная величина
* распределение явно не нормальное
* много нулей
* heavy tail

❗ t-test плохо подходит  
❗ лучше использовать **Mann–Whitney U test**

# 12. Добавляем группу к user_activity

In [12]:
# Добавляем информацию о группе
user_activity = user_activity.merge(
    user_groups,
    on="user_id",
    how="left"
)

display(user_activity)

,user_id,n_likes,exp_group
0,200,1,test
1,201,3,test
2,202,2,test
3,212,4,test
4,213,7,test
...,...,...,...
65004,168538,5,test
65005,168541,2,control
65006,168544,0,test
65007,168545,5,test


# 13. Метрика 1 — доля пользователей с ≥1 лайком по группам

In [13]:
# Добавляем бинарный признак
user_activity["has_like"] = (user_activity["n_likes"] > 0).astype(int)

# Считаем доли
group_share = (
    user_activity
    .groupby("exp_group")["has_like"]
    .mean()
)

display(group_share)

exp_group
control    0.891314
test       0.898160
Name: has_like, dtype: float64

# Статистический тест для долей (z-test)

In [14]:
# Подготовка данных
success = user_activity.groupby("exp_group")["has_like"].sum().values
total = user_activity.groupby("exp_group")["has_like"].count().values

# z-test
z_stat, p_value = proportions_ztest(count=success, nobs=total)

print("Z-test для доли пользователей с ≥1 лайком")
print(f"z-stat = {z_stat:.4f}")
print(f"p-value = {p_value:.6f}")

Z-test для доли пользователей с ≥1 лайком
z-stat = -2.8440
p-value = 0.004455


# Результаты:

```
control = 0.8913
test    = 0.8982
p-value = 0.004455
```

🔎 Разница ≈ **+0.0068 (0.68 п.п.)**  
📉 p-value < 0.05

**Различие статистически значимо.**

# 14. Метрика 2 — число лайков на пользователя

In [15]:
control_likes = user_activity[user_activity["exp_group"] == "control"]["n_likes"]
test_likes = user_activity[user_activity["exp_group"] == "test"]["n_likes"]

u_stat, p_value_mw = mannwhitneyu(
    control_likes,
    test_likes,
    alternative="two-sided"
)

print("Mann–Whitney U test для числа лайков")
print(f"U-stat = {u_stat:.4f}")
print(f"p-value = {p_value_mw:.6f}")

Mann–Whitney U test для числа лайков
U-stat = 518358073.0000
p-value = 0.000030


# Результаты:

```
p-value = 0.000030
```

**Различие статистически значимо.**

# 15. Финальная интерпретация результатов

In [16]:
alpha = 0.05

print("=== Интерпретация результатов ===\n")

print("1. Доля пользователей с ≥1 лайком:")
print(f"   p-value = {p_value:.6f}")

if p_value < alpha:
    print("   -> Различие статистически значимо.")
else:
    print("   -> Статистически значимого различия нет.")

print("\n2. Число лайков на пользователя:")
print(f"   p-value = {p_value_mw:.6f}")

if p_value_mw < alpha:
    print("   -> Различие статистически значимо.")
else:
    print("   -> Статистически значимого различия нет.")

=== Интерпретация результатов ===

1. Доля пользователей с ≥1 лайком:
   p-value = 0.004455
   -> Различие статистически значимо.

2. Число лайков на пользователя:
   p-value = 0.000030
   -> Различие статистически значимо.


# Главный смысл

> Стали ли люди ставить больше лайков в тестовой группе?

Вывод:

- Доля пользователей с хотя бы одним лайком выше в test
- Число лайков на пользователя статистически выше в test
- Различия статистически значимы при α = 0.05

Значит: **Тестовая модель приводит к росту вовлечённости пользователей.**

# 16. Приводим recommendations к списку post_id

Сейчас они строкой вида:
```
[3644 4529 4704 5294 4808]
```
Нужно превратить в список чисел.

In [17]:
# Преобразуем строку рекомендаций в список post_id
def parse_recommendations(rec_str):
    rec_str = rec_str.strip("[]")
    return list(map(int, rec_str.split()))

views["rec_list"] = views["recommendations"].apply(parse_recommendations)

display(views[["recommendations", "rec_list"]])

,recommendations,rec_list
0,[3644 4529 4704 5294 4808],"[3644, 4529, 4704, 5294, 4808]"
1,[1399 1076 797 7015 5942],"[1399, 1076, 797, 7015, 5942]"
2,[2315 3037 1861 6567 4093],"[2315, 3037, 1861, 6567, 4093]"
3,[2842 1949 162 1588 6794],"[2842, 1949, 162, 1588, 6794]"
4,[2395 5881 5648 3417 673],"[2395, 5881, 5648, 3417, 673]"
...,...,...
193290,[1733 6834 4380 1915 1627],"[1733, 6834, 4380, 1915, 1627]"
193291,[2454 191 3873 6404 1588],"[2454, 191, 3873, 6404, 1588]"
193292,[1368 1709 1616 798 5305],"[1368, 1709, 1616, 798, 5305]"
193293,[5895 6984 1978 6548 6106],"[5895, 6984, 1978, 6548, 6106]"


# 17. Готовим таблицы

In [18]:
views_small = views[["user_id", "timestamp", "rec_list"]].copy()
likes_small = likes.copy()

display(views_small,
        likes_small)

,user_id,timestamp,rec_list
0,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]"
1,146885,1654030811,"[1399, 1076, 797, 7015, 5942]"
2,50948,1654030825,"[2315, 3037, 1861, 6567, 4093]"
3,37703,1654030826,"[2842, 1949, 162, 1588, 6794]"
4,14661,1654030829,"[2395, 5881, 5648, 3417, 673]"
...,...,...,...
193290,158267,1655240340,"[1733, 6834, 4380, 1915, 1627]"
193291,63527,1655240347,"[2454, 191, 3873, 6404, 1588]"
193292,52169,1655240354,"[1368, 1709, 1616, 798, 5305]"
193293,142402,1655240373,"[5895, 6984, 1978, 6548, 6106]"


,user_id,post_id,timestamp
0,128381,4704,1654030804
1,146885,1399,1654030816
2,50948,2315,1654030828
3,14661,673,1654030831
4,37703,1588,1654030833
...,...,...,...
230171,31851,5964,1655243535
230172,51512,1498,1655243537
230173,34017,5009,1655243573
230174,13267,1787,1655243692


# 18. Делаем join по user_id

In [19]:
joined = views_small.merge(
    likes_small,
    on="user_id",
    how="left",
    suffixes=("_view", "_like")
)

display(joined)

,user_id,timestamp_view,rec_list,post_id,timestamp_like
0,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",4704.0,1.654031e+09
1,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",5294.0,1.654031e+09
2,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",3608.0,1.655049e+09
3,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",2542.0,1.655049e+09
4,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",4165.0,1.655053e+09
...,...,...,...,...,...
1017168,52169,1655240354,"[1368, 1709, 1616, 798, 5305]",1709.0,1.655240e+09
1017169,142402,1655240373,"[5895, 6984, 1978, 6548, 6106]",6548.0,1.655240e+09
1017170,72259,1655240388,"[6117, 1255, 6567, 3587, 3811]",1712.0,1.654305e+09
1017171,72259,1655240388,"[6117, 1255, 6567, 3587, 3811]",5070.0,1.654309e+09


# 19. Фильтруем по времени

1 час = 3600 секунд

In [20]:
# Лайк произошёл после показа
cond_time_after = joined["timestamp_like"] >= joined["timestamp_view"]

# Лайк не позднее 1 часа
cond_time_window = (
    joined["timestamp_like"] - joined["timestamp_view"] <= 3600
)

filtered = joined[cond_time_after & cond_time_window].copy()
display(filtered)

,user_id,timestamp_view,rec_list,post_id,timestamp_like
0,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",4704.0,1.654031e+09
1,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",5294.0,1.654031e+09
7,146885,1654030811,"[1399, 1076, 797, 7015, 5942]",1399.0,1.654031e+09
11,50948,1654030825,"[2315, 3037, 1861, 6567, 4093]",2315.0,1.654031e+09
16,37703,1654030826,"[2842, 1949, 162, 1588, 6794]",1588.0,1.654031e+09
...,...,...,...,...,...
1017160,119630,1655240337,"[3143, 599, 4588, 1577, 7077]",599.0,1.655240e+09
1017163,158267,1655240340,"[1733, 6834, 4380, 1915, 1627]",6834.0,1.655240e+09
1017167,63527,1655240347,"[2454, 191, 3873, 6404, 1588]",3873.0,1.655240e+09
1017168,52169,1655240354,"[1368, 1709, 1616, 798, 5305]",1709.0,1.655240e+09


# 20. Проверяем, был ли лайк в рекомендациях

In [21]:
# Проверяем, входит ли post_id лайка в список рекомендаций

filtered["is_recommended"] = filtered.apply(
    lambda row: row["post_id"] in row["rec_list"],
    axis=1
)

filtered = filtered[filtered["is_recommended"]]
display(filtered)

,user_id,timestamp_view,rec_list,post_id,timestamp_like,is_recommended
0,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",4704.0,1.654031e+09,True
1,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",5294.0,1.654031e+09,True
7,146885,1654030811,"[1399, 1076, 797, 7015, 5942]",1399.0,1.654031e+09,True
11,50948,1654030825,"[2315, 3037, 1861, 6567, 4093]",2315.0,1.654031e+09,True
16,37703,1654030826,"[2842, 1949, 162, 1588, 6794]",1588.0,1.654031e+09,True
...,...,...,...,...,...,...
1017160,119630,1655240337,"[3143, 599, 4588, 1577, 7077]",599.0,1.655240e+09,True
1017163,158267,1655240340,"[1733, 6834, 4380, 1915, 1627]",6834.0,1.655240e+09,True
1017167,63527,1655240347,"[2454, 191, 3873, 6404, 1588]",3873.0,1.655240e+09,True
1017168,52169,1655240354,"[1368, 1709, 1616, 798, 5305]",1709.0,1.655240e+09,True


# 21. Агрегируем к уровню показа

Теперь нам нужно получить таблицу:
```
(user_id, timestamp_view) → количество лайков
```

In [22]:
likes_per_view = (
    filtered
    .groupby(["user_id", "timestamp_view"])
    .size()
    .reset_index(name="n_likes_view")
)

display(likes_per_view)

,user_id,timestamp_view,n_likes_view
0,200,1655009047,1
1,201,1654444704,1
2,201,1654664039,2
3,202,1654480554,2
4,212,1654102755,3
...,...,...,...
137866,168541,1655016237,2
137867,168545,1654050845,2
137868,168545,1654295281,1
137869,168545,1654412184,1


# 22. Возвращаем показы без лайков

Очень важно — нам нужны ВСЕ показы

In [23]:
all_views = views_small.rename(columns={"timestamp": "timestamp_view"})

views_with_likes = all_views.merge(
    likes_per_view,
    on=["user_id", "timestamp_view"],
    how="left"
)

views_with_likes["n_likes_view"] = views_with_likes["n_likes_view"].fillna(0)

display(views_with_likes)

,user_id,timestamp_view,rec_list,n_likes_view
0,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",2.0
1,146885,1654030811,"[1399, 1076, 797, 7015, 5942]",1.0
2,50948,1654030825,"[2315, 3037, 1861, 6567, 4093]",1.0
3,37703,1654030826,"[2842, 1949, 162, 1588, 6794]",2.0
4,14661,1654030829,"[2395, 5881, 5648, 3417, 673]",1.0
...,...,...,...,...
193290,158267,1655240340,"[1733, 6834, 4380, 1915, 1627]",1.0
193291,63527,1655240347,"[2454, 191, 3873, 6404, 1588]",1.0
193292,52169,1655240354,"[1368, 1709, 1616, 798, 5305]",1.0
193293,142402,1655240373,"[5895, 6984, 1978, 6548, 6106]",1.0


# 23. Считаем hitrate

In [24]:
views_with_likes["has_click"] = (views_with_likes["n_likes_view"] > 0).astype(int)

hitrate = views_with_likes["has_click"].mean()

print(f"Hitrate (доля показов с ≥1 лайком): {hitrate:.4f}")

Hitrate (доля показов с ≥1 лайком): 0.7133


# Что сейчас посчитали

Это:
> Вероятность того, что показ рекомендаций привёл к клику


# 24. Добавим exp_group обратно к показам

In [25]:
# Добавляем группу к показам
views_with_likes = views_with_likes.merge(
    views[["user_id", "timestamp", "exp_group"]],
    left_on=["user_id", "timestamp_view"],
    right_on=["user_id", "timestamp"],
    how="left"
)

views_with_likes.drop(columns=["timestamp"], inplace=True)

display(views_with_likes)

,user_id,timestamp_view,rec_list,n_likes_view,has_click,exp_group
0,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",2.0,1,control
1,146885,1654030811,"[1399, 1076, 797, 7015, 5942]",1.0,1,test
2,50948,1654030825,"[2315, 3037, 1861, 6567, 4093]",1.0,1,test
3,37703,1654030826,"[2842, 1949, 162, 1588, 6794]",2.0,1,test
4,14661,1654030829,"[2395, 5881, 5648, 3417, 673]",1.0,1,test
...,...,...,...,...,...,...
193290,158267,1655240340,"[1733, 6834, 4380, 1915, 1627]",1.0,1,test
193291,63527,1655240347,"[2454, 191, 3873, 6404, 1588]",1.0,1,control
193292,52169,1655240354,"[1368, 1709, 1616, 798, 5305]",1.0,1,test
193293,142402,1655240373,"[5895, 6984, 1978, 6548, 6106]",1.0,1,test


# 25. Создаём 100 бакетов

Будем использовать hash user_id

In [26]:
def get_bucket(user_id, n_buckets=100):
    return int(hashlib.md5(str(user_id).encode()).hexdigest(), 16) % n_buckets

views_with_likes["bucket"] = views_with_likes["user_id"].apply(get_bucket)

display(views_with_likes)

,user_id,timestamp_view,rec_list,n_likes_view,has_click,exp_group,bucket
0,128381,1654030803,"[3644, 4529, 4704, 5294, 4808]",2.0,1,control,64
1,146885,1654030811,"[1399, 1076, 797, 7015, 5942]",1.0,1,test,33
2,50948,1654030825,"[2315, 3037, 1861, 6567, 4093]",1.0,1,test,77
3,37703,1654030826,"[2842, 1949, 162, 1588, 6794]",2.0,1,test,60
4,14661,1654030829,"[2395, 5881, 5648, 3417, 673]",1.0,1,test,23
...,...,...,...,...,...,...,...
193290,158267,1655240340,"[1733, 6834, 4380, 1915, 1627]",1.0,1,test,61
193291,63527,1655240347,"[2454, 191, 3873, 6404, 1588]",1.0,1,control,74
193292,52169,1655240354,"[1368, 1709, 1616, 798, 5305]",1.0,1,test,21
193293,142402,1655240373,"[5895, 6984, 1978, 6548, 6106]",1.0,1,test,78


# 26. Считаем hitrate по бакетам

In [27]:
bucket_stats = (
    views_with_likes
    .groupby(["exp_group", "bucket"])
    .agg(
        total_views=("has_click", "count"),
        total_clicks=("has_click", "sum")
    )
    .reset_index()
)

bucket_stats["bucket_ctr"] = (
    bucket_stats["total_clicks"] / bucket_stats["total_views"]
)

display(bucket_stats)

,exp_group,bucket,total_views,total_clicks,bucket_ctr
0,control,0,1047,735,0.702006
1,control,1,987,688,0.697062
2,control,2,1026,730,0.711501
3,control,3,969,696,0.718266
4,control,4,903,638,0.706534
...,...,...,...,...,...
195,test,95,899,635,0.706340
196,test,96,971,698,0.718847
197,test,97,954,714,0.748428
198,test,98,995,715,0.718593


# 27. Разделяем контроль и тест

In [28]:
control_buckets = bucket_stats[
    bucket_stats["exp_group"] == "control"
]["bucket_ctr"]

test_buckets = bucket_stats[
    bucket_stats["exp_group"] == "test"
]["bucket_ctr"]

# 28. t-test по бакетам

Теперь можно применять t-test, потому что:
- бакеты независимы
- всего 100 наблюдений

In [29]:
t_stat, p_value_bucket = ttest_ind(
    control_buckets,
    test_buckets,
    equal_var=False  # safer вариант
)

print("Bucket-based t-test для hitrate")
print(f"t-stat = {t_stat:.4f}")
print(f"p-value = {p_value_bucket:.6f}")

Bucket-based t-test для hitrate
t-stat = -6.0062
p-value = 0.000000


# Интерпретация

Если: `p-value < 0.05` → различие статистически значимо

Если: `p-value >= 0.05` → статистически значимого различия нет

In [30]:
group_hitrate = (
    views_with_likes
    .groupby("exp_group")["has_click"]
    .mean()
)

display(group_hitrate)

uplift = group_hitrate["test"] - group_hitrate["control"]
print(f"Разница hitrate: {uplift:.4f} ({uplift*100:.2f} п.п.)")

exp_group
control    0.706655
test       0.719843
Name: has_click, dtype: float64

Разница hitrate: 0.0132 (1.32 п.п.)


# Финальный вывод A/B эксперимента

User-level метрики:  
✔ Доля пользователей с ≥1 лайком выросла  
✔ Число лайков на пользователя выросло  

View-level метрика:  
✔ Hitrate (CTR) статистически значимо выше в test  

Итог: **тестовая модель улучшает вовлечённость пользователей.**